<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/tf_premade_estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install tensorflow==2.0.0-alpha0

     |████████████████████████████████| 79.9MB 420kB/s 
     |████████████████████████████████| 419kB 47.6MB/s 
     |████████████████████████████████| 3.0MB 34.5MB/s 
  Found existing installation: tf-estimator-nightly 1.14.0.dev2019060501
    Uninstalling tf-estimator-nightly-1.14.0.dev2019060501:
      Successfully uninstalled tf-estimator-nightly-1.14.0.dev2019060501
  Found existing installation: tb-nightly 1.14.0a20190603
    Uninstalling tb-nightly-1.14.0a20190603:
      Successfully uninstalled tb-nightly-1.14.0a20190603
  Found existing installation: tensorflow 2.0.0b0
    Uninstalling tensorflow-2.0.0b0:
      Successfully uninstalled tensorflow-2.0.0b0


In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


In [3]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [4]:
!ls

baseline_model	data  sample_data


In [5]:
!wget -P /content/data https://storage.googleapis.com/tf-datasets/titanic/train.csv

--2019-06-13 08:28:45--  https://storage.googleapis.com/tf-datasets/titanic/train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 2607:f8b0:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30874 (30K) [application/octet-stream]
Saving to: ‘/content/data/train.csv’

train.csv           100%[===================>]  30.15K  --.-KB/s    in 0s      

2019-06-13 08:28:45 (88.8 MB/s) - ‘/content/data/train.csv’ saved [30874/30874]



In [6]:
!wget -P /content/data https://storage.googleapis.com/tf-datasets/titanic/eval.csv

--2019-06-13 08:28:49--  https://storage.googleapis.com/tf-datasets/titanic/eval.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 2607:f8b0:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13049 (13K) [application/octet-stream]
Saving to: ‘/content/data/eval.csv’

eval.csv            100%[===================>]  12.74K  --.-KB/s    in 0s      

2019-06-13 08:28:49 (88.6 MB/s) - ‘/content/data/eval.csv’ saved [13049/13049]



In [5]:
train_file = "./data/train.csv"
eval_file = "./data/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

   survived     sex   age  ...     deck  embark_town  alone
0         0    male  22.0  ...  unknown  Southampton      n
1         1  female  38.0  ...        C    Cherbourg      n
2         1  female  26.0  ...  unknown  Southampton      y
3         1  female  35.0  ...        C  Southampton      n
4         0    male  28.0  ...  unknown   Queenstown      y

[5 rows x 10 columns]
   survived     sex   age  ...     deck  embark_town  alone
0         0    male  35.0  ...  unknown  Southampton      y
1         0    male  54.0  ...        E  Southampton      y
2         1  female  58.0  ...        C  Southampton      y
3         1  female  55.0  ...  unknown  Southampton      y
4         1    male  34.0  ...        D  Southampton      y

[5 rows x 10 columns]


In [6]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(eval_df.head())
print(y_train.head())
print(y_eval.head())

      sex   age  n_siblings_spouses  parch  ...  class     deck  embark_town alone
0    male  22.0                   1      0  ...  Third  unknown  Southampton     n
1  female  38.0                   1      0  ...  First        C    Cherbourg     n
2  female  26.0                   0      0  ...  Third  unknown  Southampton     y
3  female  35.0                   1      0  ...  First        C  Southampton     n
4    male  28.0                   0      0  ...  Third  unknown   Queenstown     y

[5 rows x 9 columns]
      sex   age  n_siblings_spouses  parch  ...   class     deck  embark_town alone
0    male  35.0                   0      0  ...   Third  unknown  Southampton     y
1    male  54.0                   0      0  ...   First        E  Southampton     y
2  female  58.0                   0      0  ...   First        C  Southampton     y
3  female  55.0                   0      0  ...  Second  unknown  Southampton     y
4    male  34.0                   0      0  ...  Second     

In [7]:
train_df.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [8]:
# 离散特征
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                       'deck', 'embark_town', 'alone']
# 连续特征
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column, vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column, vocab)))
    
for categorical_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            categorical_column, dtype = tf.float32))

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [0]:
def make_dataset(data_df, label_df, epochs=10, shuffle=True,
                 batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [10]:
output_dir = 'baseline_model'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
baseline_estimator = tf.estimator.BaselineClassifier(
    model_dir = output_dir,
    n_classes = 2)
baseline_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs=100))

W0613 08:32:34.579780 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:238: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0613 08:32:34.691857 139621525448576 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/variable_scope.py:883: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0613 08:32:34.710655 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/head/base_head.py:546: to_float (from tensorf

In [11]:
baseline_estimator.evaluate(input_fn=lambda:make_dataset(
    eval_df, y_eval, epochs=1, shuffle=False, batch_size=20))

W0613 08:33:38.951739 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


{'accuracy': 0.625,
 'accuracy_baseline': 0.625,
 'auc': 0.5,
 'auc_precision_recall': 0.375,
 'average_loss': 0.66182464,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.65851253,
 'precision': 0.0,
 'prediction/mean': 0.38611102,
 'recall': 0.0}

In [12]:
linear_output_dir = 'linear_model'
if not os.path.exists(linear_output_dir):
    os.mkdir(linear_output_dir)
linear_estimator = tf.estimator.LinearClassifier(
    model_dir = linear_output_dir,
    n_classes = 2,
    feature_columns = feature_columns)

linear_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs=100))

W0613 08:36:08.682493 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/lookup_ops.py:1347: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0613 08:36:08.689199 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4307: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0613 08:36:08.690248 139621525448576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4362: VocabularyListCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is depre

In [14]:
linear_estimator.evaluate(input_fn=lambda:make_dataset(
    eval_df, y_eval, epochs=1, shuffle=False))

{'accuracy': 0.78409094,
 'accuracy_baseline': 0.625,
 'auc': 0.8363942,
 'auc_precision_recall': 0.7823725,
 'average_loss': 0.47048414,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.45538086,
 'precision': 0.73333335,
 'prediction/mean': 0.35489297,
 'recall': 0.6666667}

In [15]:
dnn_output_dir = './dnn_model'
if not os.path.exists(dnn_output_dir):
    os.mkdir(dnn_output_dir)
    
dnn_estimator = tf.estimator.DNNClassifier(
    model_dir = dnn_output_dir,
    n_classes = 2,
    feature_columns = feature_columns,
    hidden_units = [128, 128],
    activation_fn = tf.nn.relu,
    optimizer = 'Adam')

dnn_estimator.train(input_fn = lambda: make_dataset(
    train_df, y_train, epochs=100))

W0613 08:39:58.529960 139621525448576 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1257: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
dnn_estimator.evaluate(input_fn = lambda: make_dataset(
    eval_df, y_eval, epochs=1, shuffle=False))

{'accuracy': 0.7765151,
 'accuracy_baseline': 0.625,
 'auc': 0.84524035,
 'auc_precision_recall': 0.7993873,
 'average_loss': 0.50703615,
 'global_step': 1960,
 'label/mean': 0.375,
 'loss': 0.48604685,
 'precision': 0.6923077,
 'prediction/mean': 0.3917805,
 'recall': 0.72727275}